In [1]:
import pickle
from importlib import reload
import numpy as np
from scipy import sparse
from zipfile import ZipFile
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import features
reload(features)
from features import load_features

In [3]:
import submission
reload(submission)
from submission import create_submission

In [4]:
with ZipFile('../data/train.csv.zip') as z:
    with z.open('train.csv') as f:
        train_df = pd.read_csv(f)
y = train_df['deal_probability'].copy()
del train_df

In [5]:
import xgboost as xgb

In [6]:
feature_names = ['image_is_null', 'item_seq_number', 'price', 'title_length_chars', 'description_length_chars',
                 'mean_encoded_categorical',
                 'title_tfidf_7000_pymorphy', 'description_tfidf_50000_pymorphy']

In [7]:
_, train_features, categorical_indices = load_features('train', feature_names)

In [8]:
train_features = sparse.hstack(train_features)

In [9]:
params = {
    'max_depth': 5,
    'learning_rate': 0.05,
    'n_estimators': 1000,
    'n_jobs': 4,
    'silent': False,
}

In [12]:
reg = xgb.XGBRegressor(max_depth=5, learning_rate=0.05, n_estimators=1000, n_jobs=4, silent=False)

In [13]:
reg.fit(train_features, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [14]:
_, test_features, _ = load_features('test', feature_names)

In [15]:
test_features = sparse.hstack(test_features)

In [16]:
with ZipFile('../data/test.csv.zip') as z:
    with z.open('test.csv') as f:
        test_df = pd.read_csv(f)

In [17]:
create_submission(reg.predict(test_features), '../submissions/', 'xgb_first', test_df)

In [20]:
reg.feature_importances_[:5]

array([0.00692441, 0.05339488, 0.09571072, 0.01723408, 0.0225428 ],
      dtype=float32)